<a href="https://colab.research.google.com/github/newneo4/Detector-de-SQL-Injection/blob/main/SQL_injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sql-injection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F538306%2F2600626%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T051552Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1af21363d0aaf0935192049cefc78ff12598f12e005791b654e1aa14ad519cd4cb4e93f565ef8474becd2a634b8126fe0fcfbed8312574697ece1ad33b088ff9df30bf79729ef1342f082dce079930bc00570e86a2e4664b8dbaf71edc516675340f68be6c0e0a3cb0dc0ed493aeb31bfd8492c05b6407650d862bae4cbaf186121f96fd7769af06dd28c427a88d434d38681c79fb588567f842507594f9e55d55e73ca3de0b7f2840a3be70634a6c35841f966eb7595f2b504324ab387fdc105e830eef0406ac2f83398bdfa452f1b1f6e433a6bf8d83aed206c915fc51a3f655e9a6c4e5b8368a366a852fffbfbcc80f7cea471b93b140d46d3b0c8cfbaa7a,sqli-xss-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3866735%2F6771498%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T051552Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D319382be2c1d5e1b2fc95a3a7533ec0ec044d3af681a04f03445c146736b3dbbc8eecabf4a0c786f55a49a29c94b2577db26312a72ce209c121cf8b55ee380f5325894efa5e5f2732328caa2c8d48ce3ec257aa258f039642340f7b542740657749f022c60bf5563f6564ba192e086e4002f42757122517864ae108cd138f37d3d622fa800c50617f19ab1bb59559571bbc85456523cc535437eddc8a11ed7fbb9cdf0292d26d8c82fe8a4a5d5a337716399144d4685a118494a32c3f3e14581218f4b31f46c307cd9c0f781e9e3dd5878f865d483486a86c902725b034129dcda9c6c4d21aed052184fbcfc47bc5da0421c977609512ec3c2c584930a02e7ea'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length}')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl}")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDescargado y descomprimido: {directory}')
    except HTTPError as e:
        print(f'Error al cargar datos {download_url} en el directorio {destination_path}')
        continue
    except OSError as e:
        print(f'Error el cargar datos {download_url} en el directorio {destination_path}')
        continue

print('Data source import complete.')


**IMPORTS**

In [ ]:

import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn import tree
from tensorflow.keras import models, layers
import warnings

from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import precision_recall_curve,precision_recall_fscore_support
import seaborn as sns
warnings.filterwarnings('ignore')

**Limpiado del dataset**

In [ ]:
df2 = pd.read_csv('/kaggle/input/sqli-xss-dataset/SQLInjection_XSS_MixDataset.1.0.0.csv')
columnas_a_eliminar = ['XSS', 'Normal']
df2 = df2.sample(10000)
df2 = df2.drop(columns=columnas_a_eliminar)
df2['SQLInjection'] = df2['SQLInjection'].astype(int)
print("Data Shape:", df2.shape)
df2.head(10)

**CARGA Y PREPROCESADO DE DATOS**

In [ ]:
path = '../input/sql-injection-dataset/'
df = pd.read_csv(path + "sqli.csv", encoding='utf-16')
print("Data Shape:", df.shape)
df.head(10)

**DATASET 1**

In [ ]:
X = df['Sentence']
y = df['Label']
print(X.shape, y.shape)
print("Dataset Input:", "\n", X.head(5))
print("Dataset Label:", "\n", y.head(5))

**DATASET 2**

In [ ]:
df2 = df2.rename(columns={'SQLInjection': 'Label'})
X2 = df2['Sentence']
y2 = df2['Label']
print(X2.shape, y2.shape)
print("Dataset Input:", "\n", X2.head(5))
print("Dataset Label:", "\n", y2.head(5))

In [ ]:
import nltk
nltk.download('stopwords')
vectorizer = CountVectorizer(min_df = 2, max_df = 0.8, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(X.values.astype('U')).toarray()

In [ ]:
import nltk
nltk.download('stopwords')
vectorizer = CountVectorizer(min_df = 2, max_df = 0.8, stop_words = stopwords.words('english'))
X2 = vectorizer.fit_transform(X2.values.astype('U')).toarray()

**DATASET 1**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

**DATASET 2**

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2)
print(X2_train.shape)
print(y2_train.shape)
print(X2_test.shape)
print(y2_test.shape)

In [ ]:
import numpy as np

# Suponiendo que 'y2_train' es tu variable objetivo en el conjunto de entrenamiento
# y2_train = ... (tus etiquetas de entrenamiento)

# Verificar la distribución de clases
clases_unicas = np.unique(y2_train)
cantidad_clases = len(clases_unicas)

print("Clases únicas presentes en y2_train:", clases_unicas)
print("Cantidad de clases:", cantidad_clases)


In [ ]:
import numpy as np

unos = np.count_nonzero(X_train)
ceros = X_train.size - unos

print("Cantidad de ceros:", ceros)
print("Cantidad de unos:", unos)

**DATASET 1**

In [ ]:
f1_dict = {}
precision_dict = {}
recall_dict = {}
accuracy_dict = {}
train_accuracy = {}
validation_accuracy = {}
test_accuracy = {}

**DATASET 2**

In [ ]:
f1_dict2 = {}
precision_dict2 = {}
recall_dict2 = {}
accuracy_dict2 = {}
train_accuracy2 = {}
validation_accuracy2 = {}
test_accuracy2 = {}

**LOGISTIC REGRESSION DATASET 1**

In [ ]:
lr_clf = LogisticRegression()
y_pred_lr = lr_clf.fit(X_train, y_train)
y_pred = y_pred_lr.predict(X_test)
print(f"Accuracy of Logistic Regression on test set : {accuracy_score(y_pred, y_test)}")
print(f"F1 Score of Logistic Regression on test set : {f1_score(y_pred, y_test)}")

f1_dict["LogisticRegression"] = f1_score(y_pred, y_test)
precision_dict["LogisticRegression"] = precision_score(y_pred, y_test)
recall_dict["LogisticRegression"] = recall_score(y_pred, y_test)
accuracy_dict['LogisticRegression'] = accuracy_score(y_pred, y_test)

**LOGISTIC REGRESSION DATASET 2**

In [ ]:
lr_clf2 = LogisticRegression()
y2_pred_lr = lr_clf2.fit(X2_train, y2_train)
y2_pred = y2_pred_lr.predict(X2_test)
print(f"Accuracy of Logistic Regression on test set : {accuracy_score(y2_pred, y2_test)}")
print(f"F1 Score of Logistic Regression on test set : {f1_score(y2_pred, y2_test)}")

f1_dict2["LogisticRegression"] = f1_score(y2_pred, y2_test)
precision_dict2["LogisticRegression"] = precision_score(y2_pred, y2_test)
recall_dict2["LogisticRegression"] = recall_score(y2_pred, y2_test)
accuracy_dict2['LogisticRegression'] = accuracy_score(y2_pred, y2_test)

**RANDOM FOREST**

**DATASET 1**

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print(f"Precision de Random Forest en el conjunto de datos : {accuracy_score(y_pred, y_test)}")
print(f"Puntuacion F1 en el conjunto de datos : {f1_score(y_pred, y_test)}")

f1_dict["RandomForest"] = f1_score(y_pred, y_test)
precision_dict["RandomForest"] = precision_score(y_pred, y_test)
recall_dict["RandomForest"] = recall_score(y_pred, y_test)
accuracy_dict['RandomForest'] = accuracy_score(y_pred, y_test)

**DATASET 2**

In [ ]:
rf_clf2 = RandomForestClassifier()
rf_clf2.fit(X2_train, y2_train)
y2_pred = rf_clf2.predict(X2_test)
print(f"Precision de Random Forest en el conjunto de datos : : {accuracy_score(y2_pred, y2_test)}")
print(f"Puntuacion F1 en el conjunto de datos : {f1_score(y2_pred, y2_test)}")

f1_dict2["RandomForest"] = f1_score(y2_pred, y2_test)
precision_dict2["RandomForest"] = precision_score(y2_pred, y2_test)
recall_dict2["RandomForest"] = recall_score(y2_pred, y2_test)
accuracy_dict2['RandomForest'] = accuracy_score(y2_pred, y2_test)

**SUPPORT VECTOR MACHINES**

**DATASET 1**

In [ ]:
svm_clf = SVC(gamma = 'auto')
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)
print(f"Precision de SVM en el conjunto de datos : {accuracy_score(y_pred, y_test)}")
print(f"Puntuacion F1 en el conjunto de datos : {f1_score(y_pred, y_test)}")

f1_dict["SVM"] = f1_score(y_pred, y_test)
precision_dict["SVM"] = precision_score(y_pred, y_test)
recall_dict["SVM"] = recall_score(y_pred, y_test)
accuracy_dict['SVM'] = accuracy_score(y_pred, y_test)

**DATASET 2**

In [ ]:
svm_clf2 = SVC(gamma = 'auto')
svm_clf2.fit(X2_train, y2_train)
y2_pred = svm_clf2.predict(X2_test)
print(f"Precision de SVM en el conjunto de datos : {accuracy_score(y2_pred, y2_test)}")
print(f"Puntuacion F1 en el conjunto de datos : {f1_score(y2_pred, y2_test)}")

f1_dict2["SVM"] = f1_score(y2_pred, y2_test)
precision_dict2["SVM"] = precision_score(y2_pred, y2_test)
recall_dict2["SVM"] = recall_score(y2_pred, y2_test)
accuracy_dict2['SVM'] = accuracy_score(y2_pred, y2_test)

**NAIVE BAYES**

**DATASET 1**

In [ ]:
nb_clf = GaussianNB()
nb_clf.fit(X_train, y_train)
y_pred = nb_clf.predict(X_test)
print(f"Precision de Naive Bayes en el conjunto de datos : {accuracy_score(y_pred, y_test)}")
print(f"Puntuacion F1 en el conjunto de datos : {f1_score(y_pred, y_test)}")

f1_dict["NaiveBayes"] = f1_score(y_pred, y_test)
precision_dict["NaiveBayes"] = precision_score(y_pred, y_test)
recall_dict["NaiveBayes"] = recall_score(y_pred, y_test)
accuracy_dict['NaiveBayes'] = accuracy_score(y_pred, y_test)

**DATASET 2**

In [ ]:
nb_clf2 = GaussianNB()
nb_clf2.fit(X2_train, y2_train)
y2_pred = nb_clf2.predict(X2_test)
print(f"Precision de Naive Bayes en el conjunto de datos : {accuracy_score(y2_pred, y2_test)}")
print(f"Puntuacion F1 en el conjunto de datos : {f1_score(y2_pred, y2_test)}")

f1_dict2["NaiveBayes"] = f1_score(y2_pred, y2_test)
precision_dict2["NaiveBayes"] = precision_score(y2_pred, y2_test)
recall_dict2["NaiveBayes"] = recall_score(y2_pred, y2_test)
accuracy_dict2['NaiveBayes'] = accuracy_score(y2_pred, y2_test)

**DECISION TREES**


**DATASET 1**

In [ ]:
DT = tree.DecisionTreeClassifier()
DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)
print(f"Precision del arbol de decision en el conjunto de datos : {accuracy_score(y_pred, y_test)}")
print(f"Puntuacion F1 en el conjunto de datos : {f1_score(y_pred, y_test)}")

f1_dict["DecisionTree"] = f1_score(y_pred, y_test)
precision_dict["DecisionTree"] = precision_score(y_pred, y_test)
recall_dict["DecisionTree"] = recall_score(y_pred, y_test)
accuracy_dict['DecisionTree'] = accuracy_score(y_pred, y_test)

**DATASET 2**

In [ ]:
DT2 = tree.DecisionTreeClassifier()
DT2.fit(X2_train, y2_train)
y2_pred = DT2.predict(X2_test)
print(f"Precision del arbol de decision en el conjunto de datos : {accuracy_score(y2_pred, y2_test)}")
print(f"Puntuacion F1 en el conjunto de datos : {f1_score(y2_pred, y2_test)}")

f1_dict2["DecisionTree"] = f1_score(y2_pred, y2_test)
precision_dict2["DecisionTree"] = precision_score(y2_pred, y2_test)
recall_dict2["DecisionTree"] = recall_score(y2_pred, y2_test)
accuracy_dict2['DecisionTree'] = accuracy_score(y2_pred, y2_test)

**CONVOLUTIONAL NEURAL NETWORK**

In [ ]:
def plot_history(history):
    history_dict = history.history
    train_loss = history_dict['loss']
    val_loss = history_dict['val_loss']
    epochs = range(1, len(history_dict['loss'])+1)
    plt.plot(epochs, train_loss,'b', label='Training error')
    plt.plot(epochs, val_loss,'b', color="orange", label='Validation error')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_historyAcc(history):
    history_dict = history.history
    train_acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    epochs = range(1, len(history_dict['accuracy'])+1)
    plt.plot(epochs, train_acc,'b', label='Training accuracy')
    plt.plot(epochs, val_acc,'b', color="orange", label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def plot_confusion_matrix(conf_matrix):
    fig, ax = plt.subplots(figsize=(7.5, 7.5))
    ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i, s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

    plt.xlabel('Predicted Value', fontsize=18)
    plt.ylabel('Actual Value', fontsize=18)
    plt.title('Confusion Matrix', fontsize=18)
    plt.show()

**DATASET 1**

In [ ]:
model = models.Sequential()
model.add(layers.Conv1D(32, 1, activation = 'relu', input_shape = (1,4717)))
model.add(layers.Conv1D(32, 1, activation = 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()
model.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

X_train1 = X_train.reshape(-1, 1, 4717)
X_test1 = X_test.reshape(-1, 1, 4717)

history2 = model.fit(X_train1, y_train, epochs = 10, validation_data = (X_test1, y_test))

In [ ]:
y_pred = model.predict(X_test1).flatten()
# y_pred1 = [1 if x>-0.5 else 0 for x in y_pred]
y_pred = np.round(y_pred)
print(f"Accuracy of CNN on test set : {accuracy_score(y_pred, y_test)}")
print(f"F1 Score of CNN on test set : {f1_score(y_pred, y_test)}")

f1_dict["CNN"] = f1_score(y_pred, y_test)
precision_dict["CNN"] = precision_score(y_pred, y_test)
recall_dict["CNN"] = recall_score(y_pred, y_test)
accuracy_dict['CNN'] = accuracy_score(y_pred, y_test)

In [ ]:
plot_model(model, show_shapes=True, to_file='CNN.png')

In [ ]:
plot_history(history2)
plot_historyAcc(history2)

In [ ]:

conf_matrix_model = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(conf_matrix_model)
print(classification_report(y_test, y_pred, target_names=["Non-Intrusion", "Intrusion"]))

**DATASET 2**

In [ ]:
model2 = models.Sequential()
model2.add(layers.Conv1D(32, 1, activation = 'relu', input_shape = (1,15321)))
model2.add(layers.Conv1D(32, 1, activation = 'relu'))
model2.add(layers.Flatten())
model2.add(layers.Dense(1, activation = 'sigmoid'))
model2.summary()
model2.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

X_train2 = X2_train.reshape(-1, 1, 15321)
X_test2 = X2_test.reshape(-1, 1, 15321)

history = model2.fit(X_train2, y2_train, epochs = 10, validation_data = (X_test2, y2_test))

In [ ]:
y2_pred = model2.predict(X_test2).flatten()
# y_pred1 = [1 if x>-0.5 else 0 for x in y_pred]
y2_pred = np.round(y2_pred)
print(f"Accuracy of CNN on test set : {accuracy_score(y2_pred, y2_test)}")
print(f"F1 Score of CNN on test set : {f1_score(y2_pred, y2_test)}")

f1_dict2["CNN"] = f1_score(y2_pred, y2_test)
precision_dict2["CNN"] = precision_score(y2_pred, y2_test)
recall_dict2["CNN"] = recall_score(y2_pred, y2_test)
accuracy_dict2['CNN'] = accuracy_score(y2_pred, y2_test)

In [ ]:
plot_model(model2, show_shapes=True, to_file='CNN2.png')

In [ ]:
plot_history(history)
plot_historyAcc(history)

In [ ]:
conf_matrix_model2 = confusion_matrix(y2_test, y2_pred)
plot_confusion_matrix(conf_matrix_model2)
print(classification_report(y2_test, y2_pred, target_names=["Non-Intrusion", "Intrusion"]))

**PLOT FINAL PARA LOS ESTADISTICOS RESULTANTES**

In [ ]:
keys2 = f1_dict, precision_dict, recall_dict, accuracy_dict
metrics = ['F1_Score', 'Precision', 'Recall', 'Accuracy']
data = pd.DataFrame(keys2)
data.index = metrics
data

In [ ]:
result = data.plot(kind='bar', rot=0, figsize=(15, 7));
result.legend(bbox_to_anchor=(1, 1.02), loc='upper left');

In [ ]:
keys3 = f1_dict2, precision_dict2, recall_dict2, accuracy_dict2
metrics2 = ['F1_Score', 'Precision', 'Recall', 'Accuracy']
data2 = pd.DataFrame(keys3)
data2.index = metrics2
data2

In [ ]:
result2 = data2.plot(kind='bar', rot=0, figsize=(15, 7));
result2.legend(bbox_to_anchor=(1, 1.02), loc='upper left');